<h2 align="center">点击下列图标在线运行HanLP</h2>
<div align="center">
	<a href="https://colab.research.google.com/github/hankcs/HanLP/blob/doc-zh/plugins/hanlp_demo/hanlp_demo/zh/amr_stl.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
	<a href="https://mybinder.org/v2/gh/hankcs/HanLP/doc-zh?filepath=plugins%2Fhanlp_demo%2Fhanlp_demo%2Fzh%2Famr_stl.ipynb" target="_blank"><img src="https://mybinder.org/badge_logo.svg" alt="Open In Binder"/></a>
</div>

## 安装

无论是Windows、Linux还是macOS，HanLP的安装只需一句话搞定：

In [ ]:
!pip install hanlp[amr] -U

## 加载模型
HanLP的工作流程是先加载模型，模型的标示符存储在`hanlp.pretrained`这个包中，按照NLP任务归类。

In [1]:
import hanlp
hanlp.pretrained.amr.ALL # 语种见名称最后一个字段或相应语料库

{'AMR3_SEQ2SEQ_BART_LARGE': 'https://file.hankcs.com/hanlp/amr/amr3_seq2seq_bart_large_83.30_20220125_114450.zip',
 'MRP2020_AMR_ENG_ZHO_XLM_BASE': 'http://download.hanlp.com/amr/extra/amr-eng-zho-xlm-roberta-base_20220412_223756.zip',
 'MRP2020_AMR_ZHO_MENGZI_BASE': 'http://download.hanlp.com/amr/extra/amr-zho-mengzi-base_20220415_101941.zip'}

调用`hanlp.load`进行加载，模型会自动下载到本地缓存。

In [2]:
amr = hanlp.load('MRP2020_AMR_ENG_ZHO_XLM_BASE')

## 抽象意义表示
抽象意义表示任务的输入为一个或多个句子，`MRP2020_AMR_ENG_ZHO_XLM_BASE`要求提供分词完毕的句子：

In [3]:
graph = amr(["男孩", "希望", "女孩", "相信", "他", "。"])

返回对象为[penman.Graph](https://penman.readthedocs.io/en/latest/api/penman.graph.html)类型：

In [4]:
graph

<AMRGraph object (top=x2) at 12603529872>

打印时为友好格式：

In [5]:
print(graph)

(x2 / 希望-01
    :arg1 (x4 / 相信-01
              :arg0 (x3 / 女孩)
              :arg1 x1)
    :arg0 (x1 / 男孩))


该AMR的可视化结果为：

![amr-zh](https://hanlp.hankcs.com/proxy/amr?tok=%E7%94%B7%E5%AD%A9%20%E5%B8%8C%E6%9C%9B%20%E5%A5%B3%E5%AD%A9%20%E7%9B%B8%E4%BF%A1%20%E4%BB%96%20%E3%80%82&language=zh&scale=1)

`MRP2020_AMR_ENG_ZHO_XLM_BASE`其实是一个Meaning Representation Parsing模型，支持输出Meaning Representation（MR）格式，该格式比AMR的表达力更强：

In [6]:
amr(["男孩", "希望", "女孩", "相信", "他", "。"], output_amr=False)

{'id': '0',
 'input': '男孩 希望 女孩 相信 他 。',
 'nodes': [{'id': 0,
   'label': '男孩',
   'anchors': [{'from': 0, 'to': 2}, {'from': 12, 'to': 13}]},
  {'id': 1, 'label': '希望-01', 'anchors': [{'from': 3, 'to': 5}]},
  {'id': 2, 'label': '女孩', 'anchors': [{'from': 6, 'to': 8}]},
  {'id': 3, 'label': '相信-01', 'anchors': [{'from': 9, 'to': 11}]}],
 'edges': [{'source': 1, 'target': 3, 'label': 'arg1'},
  {'source': 1, 'target': 0, 'label': 'arg0'},
  {'source': 3, 'target': 2, 'label': 'arg0'},
  {'source': 3, 'target': 0, 'label': 'arg1'}],
 'tops': [1],
 'framework': 'amr'}

注意上面“男孩”有2个anchor，分别对应“男孩”和“他”。也就是说，MR格式其实包含了指代消解的结果。

## 多语种支持
`MRP2020_AMR_ENG_ZHO_XLM_BASE`同时还是一个Cross-Lingual模型，支持的语言列表：

In [7]:
amr.config.frameworks

[['amr', 'eng'], ['amr', 'zho']]

用户可以通过指定language参数来实现英文抽象意义表示的分析：

In [8]:
print(amr(['The', 'boy', 'wants', 'the', 'girl', 'to', 'believe', 'him', '.'], language='eng'))

(w1 / wants-01
    :arg1 (b2 / believe-01
              :arg0 (g1 / girl)
              :arg1 b1)
    :arg0 (b1 / boy))


为了达到最佳效果，建议同时提供每个词的词干：

In [9]:
print(amr([('The', 'the'), ('boy', 'boy'), ('wants', 'want'), ('the', 'the'), ('girl', 'girl'), ('to', 'to'),
              ('believe', 'believe'), ('him', 'he'), ('.', '.')], language='eng'))

(w1 / want-01
    :arg1 (b2 / believe-01
              :arg0 (g1 / girl)
              :arg1 b1)
    :arg0 (b1 / boy))


该AMR的可视化结果为：

![amr-en](https://hanlp.hankcs.com/proxy/amr?tok=The%20boy%20wants%20the%20girl%20to%20believe%20him%20.&language=en&scale=1.)